[DIY Covid-19 Dashboard Kit](https://github.com/fsavundra/Projects) (C) Freddie Savundra, 2020 ([f.savundra@se20.qmul.ac.uk](mailto:f.savundra@se20.qmul.ac.uk) - [web](https://www.linkedin.com/in/freddie-savundra-729679131/?originalSubdomain=uk)). All rights reserved.

# Covid-19 Dashboard

The dashboard attached below references two statistics that have been examined in this dashboard. In the first instance, we examine the relationship between the number of hospital cases and the number of deaths over a 28 day period. In the second instance, we examine the number of hospital admissions and the relationship between the number of ventilators issued.

In [50]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [51]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Relationship between the number of cases and death rates

In [52]:
# Get the age distribution and save it at agedistribution.json
def getAgeDistribution():
    filters = [
        'areaType=nation',
        'areaName=England'
    ]
    # values here are the names of the PHE metrics
    structure = {
         "date": "date",
         "admissions": "newAdmissions",
         "ventilators": "covidOccupiedMVBeds"
    }
    api = Cov19API(filters=filters, structure=structure)
    agedistribution=api.get_json()
    with open("agedistribution.json", "wt") as OUTF:
        json.dump(agedistribution, OUTF)

In [53]:
# Get the timeSeries and save it at timeseries.json
def getTimeSeries():
    filters = [
        'areaType=nation',
        'areaName=England'
    ]
    structure = {
        "date": "date",
        "cases": "hospitalCases",
        "deaths": "newDeaths28DaysByDeathDate"    
    }
    api = Cov19API(filters=filters, structure=structure)
    timeseries=api.get_json()
    with open("timeseries.json", "wt") as OUTF:
        json.dump(timeseries, OUTF)

In [54]:
# Convert a date string into a pandas datetime object
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [55]:
# Process the data and sorting the dates based on timeseries.json value.
def processTimeSeriesData():
    
    with open("timeseries.json", "rt") as INFILE:
        data=json.load(INFILE)
        
    print("timeseries.json Last Update: ")
    print(data['lastUpdate']);
    
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'deaths',])
    
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['cases', 'deaths']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(timeseriesdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                timeseriesdf.loc[date, column]=value
            
    # fill in any remaining "holes" due to missing dates
    timeseriesdf.fillna(0.0, inplace=True)
    timeseriesdf.to_pickle("timeseriesdf.pkl")

In [56]:
# Process the data and sorting the dates based on agedistribution.json value.
def processAgeDistributionData():
    with open("agedistribution.json", "rt") as INFILE:
        data=json.load(INFILE)
    
    print("agedistribution.json Last Update: ")
    print(data['lastUpdate']);
    
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    agedistributiondf=pd.DataFrame(index=index, columns=['admissions', 'ventilators'])
    
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['admissions', 'ventilators']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(agedistributiondf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                agedistributiondf.loc[date, column]=value
            
    # fill in any remaining "holes" due to missing dates
    agedistributiondf.fillna(0.0, inplace=True)
    agedistributiondf.to_pickle("agedistributiondf.pkl")

In [57]:
# our API access function. This will be called by the button when it is clicked
def access_api(button):
    # Ignore the parameter, put code for polling the API here
    pleaseWait1.value = "Please wait while I reach out to the Government Statistics Page..."
    getTimeSeries()
    processTimeSeriesData()
    refreshUpdate1.value = "...The Dashboard has successfully refreshed."
    # keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
    graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})
    display(controls, graph)
    
# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets

pleaseWait1 = wdg.Label()
refreshUpdate1 = wdg.Label()
display(apibutton)
display(pleaseWait1)
display(refreshUpdate1)

getTimeSeries()
processTimeSeriesData()

timeseriesdf = pd.read_pickle("timeseriesdf.pkl")

series=wdg.SelectMultiple(
    options=['cases', 'deaths'],
    value=['cases', 'deaths'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    
    ncols=len(gcols)    
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})
display(controls, graph)

Button(description='Refresh', icon='download', style=ButtonStyle(), tooltip='Click to download current Public …

Label(value='')

Label(value='')

timeseries.json Last Update: 
2020-11-26T17:09:03.000000Z


Output()

## Relationship between Hospital Admissions and Ventilators Issued

In [58]:
# our API access function. This will be called by the button when it is clicked
def access_api(button):
    # Ignore the parameter, put code for polling the API here
    pleaseWait2.value = "Please wait while I reach out to the Government Statistics Page..."
    getAgeDistribution()
    processAgeDistributionData()
    refreshUpdate2.value = "...The Dashboard has successfully refreshed."
    graph=wdg.interactive_output(agedistribution_graph, {'gcols': series, 'gscale': scale})
    display(controls, graph)
    
# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets

pleaseWait2 = wdg.Label()
refreshUpdate2 = wdg.Label()
display(apibutton)
display(pleaseWait2)
display(refreshUpdate2)

getAgeDistribution()
processAgeDistributionData()
agedistributiondf=pd.read_pickle("agedistributiondf.pkl")
series=wdg.SelectMultiple(
    options=['admissions', 'ventilators'],
    value=['admissions', 'ventilators'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def agedistribution_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        agedistributiondf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(agedistribution_graph, {'gcols': series, 'gscale': scale})
display(controls, graph)

Button(description='Refresh', icon='download', style=ButtonStyle(), tooltip='Click to download current Public …

Label(value='')

Label(value='')

agedistribution.json Last Update: 
2020-11-26T17:09:03.000000Z


Output()

[DIY Covid-19 Dashboard Kit](https://github.com/fsavundra/Projects) (C) Freddie Savundra, 2020 ([f.savundra@se20.qmul.ac.uk](mailto:f.savundra@se20.qmul.ac.uk) - [web](https://www.linkedin.com/in/freddie-savundra-729679131/?originalSubdomain=uk)). All rights reserved. Based on UK Government data published by Public Health England.